<a href="https://colab.research.google.com/github/MatthewYancey/Capitol_Faces/blob/main/src/save_faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Save Faces
This notebook takes the facelocations and saves off jpegs of those faces for quick access.

In [1]:
!pip install ffmpeg-python

In [2]:
import pandas as pd
import sys
import os
import glob
import cv2

from google.colab import drive
drive.mount('/content/gdrive')
sys.path.append('/content/gdrive/MyDrive/repos/Capitol_Faces/src')
from helpers import general as gen

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
data_path = '/content/gdrive/MyDrive/repos/Capitol_Faces/data/'
df_faces = pd.read_csv(data_path + 'df_faces.csv')
df_faces = df_faces.loc[df_faces['frame'].notnull(), ['id', 'vid_id', 'frame', 'position']]
print(f'Number of faces in dataframe: {df_faces.shape[0]}')

Number of faces in dataframe: 2664


In [4]:
face_list = glob.glob(data_path + 'faces/*')
face_list = [int(os.path.basename(f)[:-4]) for f in face_list]
print(f'Number of faces saved: {len(face_list)}')

df_faces = df_faces.loc[~df_faces['id'].isin(face_list), :]
print(f'Number of faces to save: {df_faces.shape[0]}')

Number of faces saved: 2657
Number of faces to save: 7


In [ ]:
# sorts the dataframe so we don't reopen vidoes again
df_faces.sort_values(['vid_id', 'frame'], ascending=True, inplace=True)

# loop through the data frame
file_name = ''
for i, row in df_faces.iterrows():

    # checks if we already have this file:
    if row['id'] not in face_list:
        print(row['id'])

        # loads a new video if we have to
        if file_name != data_path + 'video/' + row['vid_id'] + '.mp4':
            file_name = data_path + 'video/' + row['vid_id'] + '.mp4'
            cap = cv2.VideoCapture(file_name)
            frame_count = 0
            ret, frame = cap.read()
        
        # checks if we need to rotate the video
        rotate = gen.rotate_value(data_path + 'video/' + row['vid_id'] + '.mp4')

        # loops through to the spot of the face
        while ret:
            if frame_count == row['frame']:
                # rotates if necissary
                if rotate:
                    frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)

                # crops and saves the image
                pos = eval(row['position'])
                frame = frame[pos[0]:pos[2], pos[3]:pos[1]]
                try:
                    cv2.imwrite(f'{data_path}faces/{row["id"]}.jpg', frame)
                    print('Saved file')
                except:
                    print(f'Bad save attempt on {row["id"]}')
                break

            frame_count += 1
            ret, frame = cap.read()